# Checker Board Solver
The following code offers an solution to the given problem located in `Description.pdf` file. Long story short, we have a checker board with some initial state. By clicking each cell, that cell and all its adjacent cells' color will be flipped. The goal is to make all cells to have the same color.

To solve this, we construct a vector _X_ which is unknown and the i-th entry will be zero if the corresponding cell in board doesn't need to be switched and one otherwise. The goal is to find this vector. By multiplying this vector by a matrix (let's say _A_) and setting each of its element to an appropriate value, this will results into a linear equation which will be solved by row reduction (also there are some tips around finding solution, because of the bounded assumptions).

## Import required libraries

In [2]:
import numpy as np

## Set problem's inputs

In [65]:
cells = [
    [1, 1, 0],
    [1, 0, 0],
    [0, 1, 0]
]

N = len(cells)
M = len(cells[0])

## Construct coefficient matrix
Construct coefficient matrix (called _A_) by setting each row appropriately. More precisely, each cell `cell(i, j)` in result matrix is sum of the following cells: 
    `cell(i, j) = cell(i, j) + cell(i, j - 1) + cell(i, j + 1) + cell(i - 1, j) + cell(i + 1, j)`

So the i-th row in matrix _A_, except these elements which are set to one, others will be set to zero.

In [82]:
def is_valid(N, M, i, j):
    return 0 <= min(i, j) and i < N and j < M

def get_corresponding_index(M, i, j):
    return i * M + j

def construct_coefficient_matrix(cells, N, M):
    A = np.zeros((N * M, N * M), dtype=int)
    
    for i in range(N):
        for j in range(M):
            if is_valid(N, M, i, j - 1):
                A[get_corresponding_index(M, i, j), get_corresponding_index(M, i, j - 1)] = 1
            if is_valid(N, M, i, j + 1):
                A[get_corresponding_index(M, i, j), get_corresponding_index(M, i, j + 1)] = 1
            if is_valid(N, M, i - 1, j):
                A[get_corresponding_index(M, i, j), get_corresponding_index(M, i - 1, j)] = 1
            if is_valid(N, M, i + 1, j):
                A[get_corresponding_index(M, i, j), get_corresponding_index(M, i + 1, j)] = 1
            A[get_corresponding_index(M, i, j), get_corresponding_index(M, i, j)] = 1

    return A

A = construct_coefficient_matrix(cells, N, M)

## Construct b vector
Construct right section of matrix equation (AX = b). By checking problem, we will find out that there are two possible _b_ vector. One is that a vector with all zero elements which is a representation of a final board with all white cells and one with all one elements. So we construct both vectors and try to find answer for each vector.

In [99]:
def construct_b_vector(N, M, cells):
    zero_b = np.ones(N * M, dtype=int)
    one_b = np.ones(N * M, dtype=int)
    
    for i in range(N):
        for j in range(M):
            if cells[i][j] == 1:
                one_b[get_corresponding_index(M, i, j)] = 0
            else:
                zero_b[get_corresponding_index(M, i, j)] = 0
    return (zero_b, one_b)

zero_b, one_b = construct_b_vector(N, M, cells)

## Construct augmented matrix
Construct augmented matrix by concatenating A to b vector.

In [100]:
def construct_augmented_matrix(A, b):
    return np.concatenate((A, b.reshape(-1, 1)), axis=1)

zero_augmented_matrix = construct_augmented_matrix(A, zero_b)
one_augmented_matrix = construct_augmented_matrix(A, one_b)

## Calculate a possible answer
Calculate a possible answer for the constructed matrix equation.

In [108]:
def compute_echelon_matrix(A):
    assert len(A.shape) == 2
    result = A.astype(int)
    
    topmost_row = 0
    for j in range(result.shape[1]):
        for i in range(topmost_row, result.shape[0]):
            if result[i][j] != 0:
                result[[topmost_row, i]] = result[[i, topmost_row]]
                break
        if result[topmost_row][j] == 0:
            # this row and all rows at below are 0 at this column
            continue
            
        # leading entry at this column is set to be at topmost_row-th,
        # so the rows at below must be scaled and then subtracted by current row
        for i in range(topmost_row + 1, result.shape[0]):
            if result[i][j] == 0:
                continue
            
            for col in range(j, result.shape[1]):
                # first scale the element, the subtract it by the candidate entry
                result[i][col] -= result[topmost_row][col]
                result[i][col] %= 2
                
        topmost_row += 1
        if topmost_row == result.shape[0]:
            break
    return result

def compute_reduced_echelon_form(A):
    assert len(A.shape) == 2
    result = compute_echelon_matrix(A)
    
    bottommost_row = result.shape[0] - 1
    while 0 <= bottommost_row:
        leftmost_column = -1 # first we assume that all of this row's elements are all zero, otherwise we update this
        for j in range(result.shape[1]):
            if result[bottommost_row][j] != 0:
                leftmost_column = j
                break
        
        if leftmost_column == -1:
            bottommost_row -= 1
            continue
        
        # then subtract rows at top, such that all entries over the current leading entry
        # become 0
        for i in range(bottommost_row):
            if result[i][leftmost_column] == 0:
                continue
            for j in range(leftmost_column, result.shape[1]):
                result[i][j] -= result[bottommost_row][j]
                result[i][j] %= 2
        bottommost_row -= 1
        
    return result

reduced_echelon_form_zero = compute_reduced_echelon_form(zero_augmented_matrix)
reduced_echelon_form_one = compute_reduced_echelon_form(one_augmented_matrix)

## Compute constructed answers
Compute constructed answers by extracting reduced echelon form matrices.

In [110]:
def extract_a_possible_answer(reduced_echelon_form, N, M):
    result = np.zeros((N, M))
    does_answer_exist = True
    for i in range(reduced_echelon_form.shape[0]):
        all_zero = True
        for j in range(reduced_echelon_form.shape[1] - 1):
            if reduced_echelon_form[i][j] != 0:
                all_zero = False
                break
        if all_zero and reduced_echelon_form[i][-1] != 0:
            does_answer_exist = False
            break
    
    if not does_answer_exist:
        result[:, :] = -1
        return result
    
    result = reduced_echelon_form[:, -1].reshape(N, M)
    return result

possible_answer_zero = extract_a_possible_answer(reduced_echelon_form_zero, N, M)
possible_answer_one = extract_a_possible_answer(reduced_echelon_form_one, N, M)

print("A possible answer to make all cells white:")
print(possible_answer_zero)
print("\nA possible answer to make all cells blue:")
print(possible_answer_one)

A possible answer to make all cells white:
[[0 1 1]
 [0 1 0]
 [0 0 0]]

A possible answer to make all cells blue:
[[1 1 0]
 [0 0 0]
 [1 0 1]]
